In [2]:
from facenet_pytorch import MTCNN
import math
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
import os
import json
import shutil

device = torch.device('cpu' if torch.cuda.is_available() else 'cuda:0')
print('Running on device: {}'.format(device))





Running on device: cpu


In [3]:
fp=open(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4\metadata.json',)
data=json.load(fp)

In [4]:
#creating data directory

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs', 'data')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data')

In [5]:
#creating train_faces directory

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data', 'train')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train')

In [6]:
#creating fake and real directories 

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train', 'real')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train\real'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train\real')
    

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train', 'fake')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train\fake'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train\fake')

In [7]:
#creating val_faces directory

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data', 'val')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val')

#creating fake and real directories 

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val', 'real')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val\real'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val\real')
    

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val', 'fake')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val\fake'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val\fake')

In [8]:
#creating validation video file

filepath = os.path.join(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04', 'validation')
if not os.path.exists(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\validation'):
    os.makedirs(r'C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\validation')

In [9]:
#training and validation split is 80:20 ratio

import random
x=(os.listdir(r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4"))

print(len(x))

amt1=0.2*len(x)
amt1=math.ceil(amt1)
print("test = ",amt1)
print("Train = ",math.ceil(0.8*len(x)))




1702
test =  341
Train =  1362


In [10]:
i=1
reqd=r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4"
dest_file=(r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_03\validation")

while i!=amt1:
    i=i+1
    source=random.choice(os.listdir(r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4"))
    src=reqd+"\\"+ source
    shutil.copy(src,dest_file)
    os.remove(src)

In [11]:
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)

In [17]:
#creating training dataset

d=r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4"



for path in os.listdir(d):
    
    #load video
    v_cap=cv2.VideoCapture(r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\dfdc_train_part_4"+'\\'+path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #looping thru video, using some frames , can modify the number of frames taken
    
    frames = []
    #v_len
    for i in tqdm(range(v_len)):
    
        # Load frame
        success = v_cap.grab()
        if i % 50 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue
        
        #add to batch
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))
        
       
        
    file_name = "real" if data[path]['label']=='REAL' else "fake"
 
    
    
    
    f=r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\train\\" + file_name + "\\"
    frames_paths = [f+ path[0:-4] + str(i) +'.jpg' for i in range(1,len(frames)+1)]
    faces = mtcnn(frames,save_path=frames_paths)



100%|██████████| 299/299 [00:01<00:00, 251.15it/s]


100%|██████████| 300/300 [00:00<00:00, 301.42it/s]


100%|██████████| 299/299 [00:01<00:00, 228.67it/s]


100%|██████████| 299/299 [00:01<00:00, 278.88it/s]
0it [00:00, ?it/s]


KeyError: 'metadata.json'

In [26]:
#creating validation dataset

d=r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\validation"



for path in os.listdir(d):
    
    #load video
    v_cap=cv2.VideoCapture(r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\validation"+'\\'+path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #looping thru video, using some frames , can modify the number of frames taken
    
    frames = []
    for i in tqdm(range(v_len)):
    
        # Load frame
        success = v_cap.grab()
        if i % 50 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue
        
        #add to batch
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))
        
    file_name = "real" if data[path]['label']=='REAL' else "fake"
    
    
    f=r"C:\Users\atmik\Desktop\sem4\DeepFakes\dfdc_train_part_04\programs\data\val\\" + file_name + "\\"
    frames_paths = [f+ path[0:-4] + str(i) +'.jpg' for i in range(1,len(frames)+1)]
    faces = mtcnn(frames,save_path=frames_paths)

In [1]:
f=open("checkpoint.pth","x")

In [2]:
f.close()

In [6]:
fp.close()